In [20]:
import torch

import torch.nn as nn

import torchvision

import torchvision.transforms as transforms

from torchvision.models import resnet18

from torch.utils.data import DataLoader

In [21]:
# Device setup (your MPS for Mac M1 will auto kick in if available)

device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
# 1. Transforms

transform = transforms.Compose([

    transforms.Resize((224,224)) ,# ResNet expects 224x224

    transforms.ToTensor(),

    transforms.Normalize(mean=[0.5]* 3, std=[0.5]* 3)
])

In [23]:
# 🐶 2. CIFAR10 (We'll simulate Dog vs Not-Dog as two classes)

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

In [24]:
# Binary classification: dog vs not-dog

def binary_target(target):

    return 1 if target == 5 else 0  # CIFAR10 class 5 is 'dog'

In [25]:
train_dataset.targets = [binary_target(t) for t in train_dataset.targets]

test_dataset.targets = [binary_target(t) for t in test_dataset.targets]

In [26]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=64)

In [27]:
# 🧠 3. Load pretrained ResNet18

model = resnet18(pretrained=True)

/opt/anaconda3/envs/masterxdl/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/masterxdl/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [28]:
# 🧊 4. Freeze all layers first

for param in model.parameters():

    param.requires_grad = False

In [29]:
# 🔓 5. Unfreeze last block (layer4)

for param in model.layer4.parameters():

    param.requires_grad = True

In [30]:
# 🧠 6. Replace the classifier

num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 1)   # Binary classification (dog vs not-dog)


model = model.to(device)

In [31]:
# 7. Loss & Optimizer

criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, model.parameters()), lr = 1e-4)

In [32]:
# 8. Training Loop

def train(model, loader, criterion, optimizer):

    model.train()

    for images , labels in loader:

        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

        outputs = model(images)

        loss = criterion(outputs, labels)


        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    print(f'Train Loss : {loss.item():.4f}')

In [33]:
# 9. Evaluation

def evaluate(model, loader):

    model.eval()

    correct, total = 0, 0


    with torch.no_grad():

        for images , labels in loader:

                images, labels = images.to(device), labels.to(device)

                outputs = torch.sigmoid(model(images)).squeeze()

                preds = (outputs > 0.5).long()

                correct += (preds == labels).sum().item()

                total += labels.size(0)

        
    print(f'Accuracy : {100 * correct / total:.2f}%')



#  10. Run Training

for epoch in range(3):
     

    print(f'Epoch {epoch +1}')

    train(model, train_loader, criterion, optimizer)

    evaluate(model, test_loader)

Epoch 1
Train Loss : 0.0557
Accuracy : 96.70%
Epoch 2
Train Loss : 0.0127
Accuracy : 96.57%
Epoch 3
Train Loss : 0.0015
Accuracy : 96.58%
